Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).

In [1]:
from sklearn import datasets
import pandas as pd
ds = datasets.load_digits()

In [2]:
ds.data

array([[  0.,   0.,   5., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,  10.,   0.,   0.],
       [  0.,   0.,   0., ...,  16.,   9.,   0.],
       ..., 
       [  0.,   0.,   1., ...,   6.,   0.,   0.],
       [  0.,   0.,   2., ...,  12.,   0.,   0.],
       [  0.,   0.,  10., ...,  12.,   1.,   0.]])

In [3]:
X = ds.data
y = ds.target

Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Эта функция реализует k-fold cross validation c k равным значению параметра cv. Мы предлагаем использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и k=5. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

1.Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1


In [8]:
from sklearn import ensemble, cross_validation, tree

In [9]:
classifier = tree.DecisionTreeClassifier()
cvs = cross_validation.cross_val_score(classifier, X, y, scoring = 'accuracy', cv = 10)

In [12]:
cvs.mean()

0.82630159178018014

2.Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [13]:
baggingClassifier = ensemble.BaggingClassifier(n_estimators = 100)
cross_validation.cross_val_score(baggingClassifier, X, y, scoring = 'accuracy', cv = 10).mean()


0.92101583675376286

In [23]:
import math
int(math.sqrt(X.shape[1]))

8

3.Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на d√ случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [26]:
baggingClassifier = ensemble.BaggingClassifier(n_estimators = 100, 
                                               max_features = int(math.sqrt(X.shape[1])))
cross_validation.cross_val_score(baggingClassifier, X, y, scoring = 'accuracy', cv = 10).mean()

0.93161641721162047

4.Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же d√ признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [39]:
dtclassifier = tree.DecisionTreeClassifier(max_features = int(math.sqrt(X.shape[1])))

baggingClassifier = ensemble.BaggingClassifier(n_estimators = 100, 
                                               base_estimator = dtclassifier)
cross_validation.cross_val_score(baggingClassifier, X, y, scoring = 'accuracy', cv = 10).mean()

0.94893881542286551

5.Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

In [37]:
rfClassifier = ensemble.RandomForestClassifier(n_estimators = 100, 
                                               max_features = int(math.sqrt(X.shape[1])))
cross_validation.cross_val_score(rfClassifier, X, y, scoring = 'accuracy', cv = 10).mean()

0.95339351423498486